In [1]:
import pandas as pd


In [2]:
df = pd.read_csv("rerank_dataset.csv", index_col=0)

df.tail(3)

,questions,answer,doc_page,doc_content,doc_source,label
90,ต้องทำอย่างไรเมื่อความเร็วรถลดลงต่ำกว่าประมาณ ...,"เมื่อความเร็วรถลดลงต่ำกว่าประมาณ 30 กม./ชม., ค...",127,แผงหน้าปัดและอุปกรณ์ควบคุม 5 \nคันสวิตช์ไฟเล...,docs/Text_PDF_.pdf,"['ลดลงตํ่ากว่า', 'E00536800021', 'งาน', 'โปรด'..."
91,ระบบ FCM อาจไม่ทำงานในสภาวะใดบ้าง?,ระบบ FCM อาจไม่ทำงานในสภาวะดังต่อไปนี้:\n- รถค...,213,○ ระบบ FCM อาจไม่ทํางานหากรถคันหน้ามีลักษณะแคบ...,docs/Text_PDF_.pdf,"['งาน', 'อาจทํา', 'ระหว่างรถ', 'อาจตรวจ', 'จับ..."
92,ตัวแสดงโหมดการขับเคลื่อน 4 ล้อแสดงสถานะการเปลี...,ตัวแสดงโหมดการขับเคลื่อน 4 ล้อจะสว่างตามตำแหน่...,180,ตัวแสดงโหมดการขับเคลื่อน 4 ล้อและตัวแสดงการทํ...,docs/Text_PDF_.pdf,"['E00606001774', 'งาน', '2WD4WD', 'จะสว่าง', '..."
